In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pareto
import sys
import pickle
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.Bayescard_BN import Bayescard_BN
from time import perf_counter
from Evaluation.utils import parse_query
from Evaluation.cardinality_estimation import parse_query_single_table

In [ ]:
def discretize_series(s, domain_size):
    n_invalid = len(s[s>=domain_size])
    s = s[s<domain_size]
    s = np.floor(s)
    new_s = np.random.randint(domain_size, size=n_invalid)
    s = np.concatenate((s, new_s))
    return np.random.permutation(s)
    
def data_generation(skew, domain_size, correlation, column_size, nrows=1000000):
    data = np.zeros((column_size, nrows))
    for i in range(column_size):
        if i == 0:
            s = np.random.randint(domain_size, size=nrows)
            data[i,:] = s
            continue
        s = pareto.rvs(b=skew, scale=1, size=nrows)
        s = discretize_series(s, domain_size)
        if i == 1:
            selected_cols = [0]
        else:
            #num_selected_cols = max(np.random.randint(int(np.ceil(i*0.1))), 1)
            num_selected_cols = 1
            selected_cols = np.random.permutation(i)[0:num_selected_cols]
        idx = np.random.permutation(nrows)[0:int(nrows*correlation)]
        if len(idx) != 0:
            selected_data = data[selected_cols, :]
            selected_data = np.ceil(np.mean(selected_data, axis=0))
            s[idx] = selected_data[idx]
        assert len(np.unique(s)) <= domain_size, "invalid domain"
        data[i,:] = s
        
    data = pd.DataFrame(data=data.transpose(), columns=[f"attr{i}" for i in range(column_size)])
    return data

def query_generation(data, table_name, num_sample=200, p=0.8, nval_per_col=4, skip_zero_bit=6):
    queries = []
    cards = []
    for i in range(num_sample):
        query, card = generate_single_query(data, table_name, p, nval_per_col, skip_zero_bit)
        while query is None:
            query, card = generate_single_query(data, table_name, p, nval_per_col, skip_zero_bit)
        queries.append(query)
        cards.append(card)
    return queries, cards

def generate_single_query(df, table_name, p=0.8, nval_per_col=4, skip_zero_bit=6):
    """
    p, nval_per_col, and skip_zero_bit are controlling the true cardinality size. As we know smaller true card 
    generally leads to larger q-error, which will bias the experimental result, so we use this to control the 
    true card to be similar for all experiments.
    """
    query = f"SELECT COUNT(*) FROM {table_name} WHERE "
    execute_query = ""
    column_names = df.columns
    n_cols = 0
    for i, col in enumerate(column_names):
        a = np.random.choice([0,1], p=[p,1-p])
        if a == 0:
            index = np.random.choice(len(df), size=nval_per_col)
            val = sorted(list(df[col].iloc[index]))
            left_val = val[0]
            right_val = val[-1]
            if left_val == right_val:
                sub_query = col + '==' + str(left_val) + ' and '
                act_sub_query = col + ' = ' + str(left_val) + ' AND '
            else:
                if skip_zero_bit:
                    left_val += skip_zero_bit
                    right_val += skip_zero_bit
                sub_query = str(left_val) + ' <= ' + col + ' <= ' + str(right_val) + ' and '
                act_sub_query = col + ' >= ' + str(left_val) + ' AND ' + col + ' <= ' + str(right_val) + ' AND '
            execute_query += sub_query
            query += act_sub_query
    if execute_query == "":
        return None,  None
    execute_query = execute_query[:-5]
    query = query[:-5]
    try:
        card = len(df.query(execute_query))
    except:
        card = 0
    if card==0:
        return None, None
    return query, card

In [ ]:
def train_one(skew, domain_size, correlation, column_size, nrows=1000000, num_sample=200, 
              p=0.8, nval_per_col=4, skip_zero_bit=6, rows_to_use=10000, n_mcv=30, n_bins=70):
    data = data_generation(skew, domain_size, correlation, column_size, nrows=nrows)
    name = f"toy_{skew}_{domain_size}_{correlation}_{column_size}"
    queries, cards = query_generation(data, name, num_sample, p, nval_per_col, skip_zero_bit)
    BN = Bayescard_BN(name)
    BN.build_from_data(data, sample_size=rows_to_use, n_mcv=n_mcv, n_bins=n_bins)
    model_path = f'/home/ziniu.wzn/BN_checkpoints/synthetic/{name}.pkl'
    pickle.dump(BN, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    BN.infer_algo = "exact-jit"
    BN.init_inference_method()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        query = parse_query_single_table(query_str.strip(), BN)
        cardinality_true = cards[query_no]
        card_start_t = perf_counter()
        cardinality_predict = BN.query(query)
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif np.isnan(cardinality_predict) or cardinality_predict == 0:
            cardinality_predict = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        elif cardinality_true == 0:
            cardinality_true = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        else:
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        latencies.append(latency_ms)
        q_errors.append(q_error)
    for i in [50, 90, 95, 99, 100]:
        print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
    print(f"average latency is {np.mean(latencies)} ms")
    return q_errors, latencies

def run_all_experiment():
    parameters_to_explore = {
        "skew": [0.1, 0.3, 0.6, 1.0, 1.5, 2.0],
        "domain_size": [10, 100, 500, 1000, 5000, 10000],
        "correlation": [0, 0.2, 0.4, 0.6, 0.8, 1.0],
        "column_num": [2, 5, 10, 50, 100, 200]
    }
    
    print("runing experiment on varying skewness: [0.1, 0.3, 0.6, 1.0, 1.3, 1.6, 2.0]}")
    print("controlled parameters are: domain_size = 100, correlation = 0.4, column_num = 10")
    skew_qerrors = []
    for skew in parameters_to_explore["skew"]:
        print("============================================================")
        print(f"Tesing skewness = {skew}")
        q_errors, latencies = train_one(skew, 100, 0.4, 10, nrows=1000000)
        skew_qerrors.append(np.asarray(q_errors))
    skew_qerrors = np.stack(skew_qerrors)
    np.save("skew_qerrors", skew_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying domain_size: [10, 100, 1000, 10000]")
    print("controlled parameters are: skewness = 1.0, correlation = 0.4, column_num = 10")
    domain_size_qerrors = []
    for domain_size in parameters_to_explore["domain_size"]:
        print("============================================================")
        print(f"Tesing domain_size = {domain_size}")
        q_errors, latencies = train_one(1, domain_size, 0.4, 10, nrows=1000000)
        domain_size_qerrors.append(np.asarray(q_errors))
    domain_size_qerrors = np.stack(domain_size_qerrors)
    np.save("domain_size_qerrors", domain_size_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying correlation: [0, 0.2, 0.4, 0.6, 0.8, 1.0]")
    print("controlled parameters are: skewness = 1.0, domain_size = 100, column_num = 10")
    correlation_qerrors = []
    for correlation in parameters_to_explore["correlation"]:
        print("============================================================")
        print(f"Tesing correlation = {correlation}")
        q_errors, latencies = train_one(1, 100, correlation, 10, nrows=1000000)
        correlation_qerrors.append(np.asarray(q_errors))
    correlation_qerrors = np.stack(correlation_qerrors)
    np.save("correlation_qerrors", correlation_qerrors)
    
    print("============================================================")
    print("============================================================")
    
    print("runing experiment on varying column_num: [2, 5, 10, 50, 100, 200]")
    print("controlled parameters are: skewness = 1.0, domain_size = 100, correlation = 0.4")
    column_num_qerrors = []
    for column_num in parameters_to_explore["column_num"]:
        print("============================================================")
        print(f"Tesing column_num = {column_num}")
        q_errors, latencies = train_one(1, 100, 0.4, column_num, nrows=1000000)
        column_num_qerrors.append(np.asarray(q_errors))
    column_num_qerrors = np.stack(column_num_qerrors)
    np.save("column_num_qerrors", column_num_qerrors)

In [ ]:
parameters_to_explore = {
        "skew": ([0.1, 0.3, 0.6, 1.0, 1.5, 2.0], [4,4,3,3,2,2]),
        "domain_size": ([10, 100, 500, 1000, 5000, 10000], [4,4,4,4,4,4]),
        "correlation": ([0, 0.2, 0.4, 0.6, 0.8, 1.0], [4,4,4,4,4,4]),
        "column_num": ([2, 5, 10, 50, 100, 200], [6,6,4,2,0,0])
    }

In [ ]:

skew = parameters_to_explore["skew"][0]
print(f"runing experiment on varying skewness: {skew}")
print("controlled parameters are: domain_size = 100, correlation = 0.4, column_num = 10")
skew_qerrors = []
for i in range(len(skew)):
    skew = parameters_to_explore["skew"][0][i]
    skip_zero_bit = parameters_to_explore["skew"][1][i]
    print("============================================================")
    print(f"Tesing skewness = {skew}")
    q_errors, latencies = train_one(skew, 100, 0.4, 10, skip_zero_bit=skip_zero_bit)
    skew_qerrors.append(np.asarray(q_errors))
skew_qerrors = np.stack(skew_qerrors)
np.save("skew_qerrors", skew_qerrors)

In [ ]:
domain_size = parameters_to_explore["domain_size"][0]
print(f"runing experiment on varying domain_size: {domain_size}")
print("controlled parameters are: skewness = 1.0, correlation = 0.4, column_num = 10")
domain_size_qerrors = []
for i in range(len(domain_size)):
    domain_size = parameters_to_explore["domain_size"][0][i]
    skip_zero_bit = parameters_to_explore["domain_size"][1][i]
    print("============================================================")
    print(f"Tesing domain_size = {domain_size}")
    q_errors, latencies = train_one(1, domain_size, 0.4, 10, nrows=1000000, skip_zero_bit=skip_zero_bit)
    domain_size_qerrors.append(np.asarray(q_errors))
domain_size_qerrors = np.stack(domain_size_qerrors)
np.save("domain_size_qerrors", domain_size_qerrors)

In [ ]:
correlation = parameters_to_explore["correlation"][0]
print(f"runing experiment on varying correlation: {correlation}")
print("controlled parameters are: skewness = 1.0, domain_size = 100, column_num = 10")
correlation_qerrors = []
for i in range(len(correlation)):
    domain_size = parameters_to_explore["correlation"][0][i]
    skip_zero_bit = parameters_to_explore["correlation"][1][i]
    print("============================================================")
    print(f"Tesing correlation = {correlation}")
    q_errors, latencies = train_one(1, 100, correlation, 10, nrows=1000000, skip_zero_bit=skip_zero_bit)
    correlation_qerrors.append(np.asarray(q_errors))
correlation_qerrors = np.stack(correlation_qerrors)
np.save("correlation_qerrors", correlation_qerrors)

In [ ]:
column_num = parameters_to_explore["column_num"][0]
print(f"runing experiment on varying column_num: {column_num}")
print("controlled parameters are: skewness = 1.0, domain_size = 100, correlation = 0.4")
column_num_qerrors = []
p_list = [0.8, 0.8, 0.8, 0.2, 0.1, 0.05]
for i in range(len(column_num)):
    column_num = parameters_to_explore["column_num"][0][i]
    skip_zero_bit = parameters_to_explore["column_num"][1][i]
    print("============================================================")
    print(f"Tesing column number = {column_num}")
    p = p_list[i]
    q_errors, latencies = train_one(1, 100, 0.4, column_num, p=p, skip_zero_bit=skip_zero_bit, nrows=1000000)
    column_num_qerrors.append(np.asarray(q_errors))
column_num_qerrors = np.stack(column_num_qerrors)
np.save("column_num_qerrors", column_num_qerrors)